In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns4
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/train.csv')
#test = pd.read_csv('/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/test.csv')
output = pd.read_csv('/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.dtypes.value_counts()

In [ ]:
#Finding the number of nan entries in each column
print(train.isnull().sum())

In [ ]:
#to see the class distribution
sns.displot(train, x="Target") #plot shows class imbalance

In [ ]:
train = train.drop(['WaterAccessPointsCalc', 'CropSpeciesVariety', 'PrimaryCropAreaSqft', 'CultivatedAreaSqft1', 'PerimeterGuardPlantsArea','CultivatedAndWildArea', 'FieldEstablishedYear', 'FarmClassification', 'FieldZoneLevel'], axis=1)


In [ ]:
#delete nan columns
train = train.dropna(axis=1, how='all')
#test = test.dropna(axis=1, how='all')

In [ ]:
#remove columns that have more missing values
n_rows= train.shape[0]

for i in train.columns:
  if train[i].isnull().sum() > 0.8*n_rows:
    train.drop(i, axis=1, inplace=True)

In [ ]:
#replacing the missing entries with median 
for i in train.columns:
  if train[i].dtype in ['int64', 'float64']:
    if train[i].isnull().sum() > 0:
      train[i].fillna(train[i].median(), inplace=True)

Finding correlation of each feature with the target class label

In [ ]:
df = train

In [ ]:
df['corr_label'] = (train['Target'] == 'high').astype(int)

In [ ]:
df['corr_label']

In [ ]:
column_to_exclude = 'corr_label'  
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.difference([column_to_exclude])

correlations = {}

for col in numerical_columns:
    correlation = df[col].corr(df['corr_label'])
    correlations[col] = correlation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

correlations_series = pd.Series(correlations)

# Plotting the correlations as a bar plot
plt.figure(figsize=(10, 6))
correlations_series.sort_values().plot(kind='barh', color='steelblue')
plt.title('Correlation between Numerical Features and High class label')
plt.xlabel('Correlation Value')
plt.ylabel('Features')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df['corr_label'] = (train['Target'] == 'low').astype(int)

In [ ]:
df['corr_label']

In [ ]:
column_to_exclude = 'corr_label'  
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.difference([column_to_exclude])

correlations = {}

for col in numerical_columns:
    correlation = df[col].corr(df['corr_label'])
    correlations[col] = correlation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

correlations_series = pd.Series(correlations)

# Plotting the correlations as a bar plot
plt.figure(figsize=(10, 6))
correlations_series.sort_values().plot(kind='barh', color='steelblue')
plt.title('Correlation between Numerical Features and Low class label')
plt.xlabel('Correlation Value')
plt.ylabel('Features')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df['corr_label'] = (train['Target'] == 'medium').astype(int)

In [ ]:
column_to_exclude = 'corr_label'  
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.difference([column_to_exclude])

correlations = {}

for col in numerical_columns:
    correlation = df[col].corr(df['corr_label'])
    correlations[col] = correlation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

correlations_series = pd.Series(correlations)

# Plotting the correlations as a bar plot
plt.figure(figsize=(10, 6))
correlations_series.sort_values().plot(kind='barh', color='steelblue')
plt.title('Correlation between Numerical Features and Medium class label')
plt.xlabel('Correlation Value')
plt.ylabel('Features')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
train.info()

In [ ]:
train = train.drop('corr_label', axis=1, errors='ignore') 


Splitting the dataset into independent and target variables

In [ ]:
X = train.drop(['Target'], axis = 1)
y = train['Target']

In [ ]:
X

In [ ]:
y

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X)

Random forest model

In [ ]:
#random forest before label encoding y
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y)

Label encoding the target variable

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
print(np.bincount(y_encoded))

In [ ]:
y_encoded

XGBoost model 

In [ ]:
from xgboost import XGBClassifier


model_xgboost = XGBClassifier(
    objective='multi:softmax', 
    num_class=3,              #since the target has 3 classes 
    learning_rate=0.1,
    max_depth=5,
    n_estimators=5000,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss'     
)

In [ ]:
model_xgboost.fit(X_train, y_encoded)

In [ ]:
import argparse
import pandas as pd
from sklearn.preprocessing import StandardScaler


def make_predictions(test_fname, predictions_fname):
    
    test = pd.read_csv(test_fname)
    
    #preprocessing test set
    test = test.drop(['WaterAccessPointsCalc', 'CropSpeciesVariety', 'PrimaryCropAreaSqft', 'CultivatedAreaSqft1', 'PerimeterGuardPlantsArea','CultivatedAndWildArea', 'FieldEstablishedYear', 'FarmClassification', 'FieldZoneLevel'], axis=1)
    test = test.dropna(axis=1, how='all')
    n_rows= test.shape[0]

    for i in test.columns:
      if test[i].isnull().sum() > 0.8*n_rows:
        test.drop(i, axis=1, inplace=True)
        
    #the mean value is significantly lower than the max value. hence, we replace with median as this dataset has extreme outliers
    for i in test.columns:
      if test[i].dtype in ['int64', 'float64']:
        if test[i].isnull().sum() > 0:
          test[i].fillna(test[i].median(), inplace=True)
    
    X_test = sc.transform(test)
    
    #Random Forest model
    rf_pred = classifier.predict(X_test)  

    #XGBoost model
    predictions = model_xgboost.predict(X_test)
    pred_labels = label_encoder.inverse_transform(predictions)
    
    #ensembling both Random Forest and XGBoosting
    predictions_df = pd.DataFrame({'xgb_pred': pred_labels, 'rf_pred': rf_pred})
    voted_pred = predictions_df.mode(axis=1)[0] 

    voted_pred = voted_pred.values

    final = pd.DataFrame()
    final['UID'] = test['UID']
    final['Target'] = voted_pred  # Use the voted predictions here

    final.to_csv('predictions_fname.csv', index=False)

    #### end code ####

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-file", type=str, help='file path of train.csv')
    parser.add_argument("--test-file", type=str, help='file path of test.csv')
    parser.add_argument("--predictions-file", type=str, help='save path of predictions')
    args = parser.parse_args()

    make_predictions(args.test_file, args.predictions_file)
